In [124]:
import numpy as np
import pandas as pd
import os
import sys
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing as pp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [125]:
#Import Data
os.chdir('E:\Academics\Research\Data\Experimental data\Compression_test\Results\As_Built')
Master_Data = pd.read_csv('Data_As_Built_New.csv').dropna()


In [126]:
# Input and output columns in Dataframe
Input = ['X','Y','Theta','Phi']
Properties = ['Yield','Modulus']
Output = Properties[1]

In [127]:
# Normalization of data
full = []
full.extend(Input)
full.extend(Properties)
Parsed_Data = Master_Data[full]
# Parsed_Data = Parsed_Data[Parsed_Data['Phi']==45]
Normalized_Data = (Parsed_Data-Parsed_Data.min())/(Parsed_Data.max()-Parsed_Data.min())
Normalized_Data = Normalized_Data.dropna()

In [128]:
IO = ['X','Y','Theta','Phi',Output]
Parsed_Data = Parsed_Data[IO]

In [129]:
Normalized_Data = (Parsed_Data-Parsed_Data.min())/(Parsed_Data.max()-Parsed_Data.min())

In [130]:
new_input = ['X','Y','Theta','Phi']

In [131]:
# Name Input and target values
X = Normalized_Data[new_input].values
y = Normalized_Data[Output].values

In [132]:
# K Fold Cross Validation

kf = KFold(n_splits=10)
mae = []
y_t=[]
y_p = []


for train, test in kf.split(X, y):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        
        
        regressor = RandomForestRegressor(n_estimators=4, criterion='mse', max_depth=None, 
                                           min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                           max_features='auto', max_leaf_nodes=None,  
                                           min_impurity_split=0, bootstrap=True, oob_score=False, n_jobs=1, 
                                           random_state=None, verbose=0, warm_start=False)
        regressor.fit(X_train,y_train)
        y_pred = regressor.predict(X_test)
        
        y_t = np.concatenate((y_t,y_test))
        y_p = np.concatenate((y_p,y_pred))
        
        mae.append(mean_absolute_error(y_test,y_pred))

In [133]:
ran = Parsed_Data[Output].max() - Parsed_Data[Output].min()
min_val = Parsed_Data[Output].min()

In [134]:
y_t = y_t*ran +min_val
y_p = y_p*ran+min_val

In [135]:
plt.plot(y_t, y_p,'b.')
plt.plot(y_t,y_t,'k--')
plt.xlabel('Actual Young\'s Modulus (GPa)',size = 15)
plt.ylabel('Predicted Young\'s Modulus (Gpa)', size = 15)
plt.title('Random Forest Regression')
plt.show()

In [136]:
mae = np.asarray(mae)

In [137]:
mae.mean()*ran

58.228571428571435